In [119]:
#sciezka z danymi
%cd "C:\Users\Dawid\Desktop\projekty\wine_quality"  

C:\Users\Dawid\Desktop\projekty\wine_quality


In [121]:
#glowne biblioteki
import pandas as pd
import numpy as np 
from sklearn.metrics import *
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras import layers
from tensorflow.keras import layers, callbacks
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

#wlasny modul
from dane.dane_wsadowe_zmiana_klas import *

In [10]:
# transformujemy Y_test i Y_train do wlasciwego formatu

Y_train_enc = np.array(Y_train_all)
Y_test_enc = np.array(Y_test)

Y_test_enc = OneHotEncoder().fit_transform(Y_test_enc.reshape(-1,1)).toarray()
Y_train_enc = OneHotEncoder().fit_transform(Y_train_enc.reshape(-1,1)).toarray()

# zmienmy jeszcze format
Y_test_enc = Y_test_enc.astype(np.float32)
Y_train_enc = Y_train_enc.astype(np.float32)

In [108]:
#definicja modelu
def model_nn(X,Y, units, drop, classNum, epoch, lr = 0.01, bn = False, pat = 10):
    '''
    X - dane treningowe w postaci array bez labeli,
    Y - labeli dla danych treningowych
    units - lista unitow dla poszczegolnych warstw
    drop - poziom dropoutu dla poszczegolnej wartswy
    classNum = ilosc klas 
    epoch - liczba epok podczas trenowania 
    lr - learning rate
    bn - czy zastosowac batch normalization 
    pat - ile interacji musi minac, aby przedwczesnie zatrzymac iterowanie
    '''
    model = tf.keras.models.Sequential()
    model.add(layers.InputLayer(input_shape = X.shape[1]))
    for i, j in enumerate(units):
        model.add(layers.Dense(j, activation='relu', name="ukryta_" + str(i)))
        model.add(layers.Dropout(drop))
        if bn == True:
            model.add(layers.BatchNormalization())
                  
    model.add(layers.Dense(classNum, name="WYJSCIE"))
    model.add(layers.Softmax())
    
    model.summary()
    
    optymalizator = tf.optimizers.Adam(lr)
    
    model.compile(optimizer=optymalizator,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['AUC'])
    
    results = model.fit(X,Y, validation_split= 0.2, epochs=epoch, verbose=1,
              callbacks=[callbacks.EarlyStopping(monitor= 'val_auc', min_delta=0.1, 
                                                    patience=pat)])
    return model

In [28]:
# wywolajmy pierwszy prosty model
model_nn1 = model_nn(X_train_all_scaled, Y_train_enc,[256],0.1,2,30)
Y_pred_nn1 = model_nn1.predict(X_test_scaled).argmax(axis = 1)

# szybka ewaluacja
eval_res = model_nn1.evaluate(X_test_scaled, Y_test_enc)
eval_names = model_nn1.metrics_names
print(f"{eval_names[0]}: {eval_res[0]}")
print(f"{eval_names[1]}: {eval_res[1]}")

#obejrzyjmy jeszcze macierz pomyłek 
matrix_nn1 = confusion_matrix(Y_test_enc.argmax(axis = 1), Y_pred_nn1)
raport_nn1 = classification_report(Y_test_enc.argmax(axis = 1), Y_pred_nn1)
print(raport_nn1)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ukryta_0 (Dense)             (None, 256)               3072      
_________________________________________________________________
dropout_27 (Dropout)         (None, 256)               0         
_________________________________________________________________
WYJSCIE (Dense)              (None, 2)                 514       
_________________________________________________________________
softmax_14 (Softmax)         (None, 2)                 0         
Total params: 3,586
Trainable params: 3,586
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
32/32 [==============================] - 1s 16ms/step - loss: 0.3419 - auc: 0.9355 - val_loss: 0.3057 - val_auc: 0.9388
Epoch 2/30
32/32 [==============================] - 0s 4ms/step - loss: 0.2818 - auc: 0.9514 - val_loss: 0.2942 - val_

In [40]:
# po kilku probach nasz najlepszy model wyglada nastepujaco
model_nn2 = model_nn(X_train_all_scaled, Y_train_enc,[1024],0.1,2,30)
Y_pred_nn2 = model_nn2.predict(X_test_scaled).argmax(axis = 1)

# szybka ewaluacja
eval_res = model_nn2.evaluate(X_test_scaled, Y_test_enc)
eval_names = model_nn2.metrics_names
print(f"{eval_names[0]}: {eval_res[0]}")
print(f"{eval_names[1]}: {eval_res[1]}")

#obejrzyjmy jeszcze macierz pomyłek 
matrix_nn2 = confusion_matrix(Y_test_enc.argmax(axis = 1), Y_pred_nn2)
raport_nn2 = classification_report(Y_test_enc.argmax(axis = 1), Y_pred_nn2)
print(raport_nn2)

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ukryta_0 (Dense)             (None, 1024)              12288     
_________________________________________________________________
dropout_44 (Dropout)         (None, 1024)              0         
_________________________________________________________________
WYJSCIE (Dense)              (None, 2)                 2050      
_________________________________________________________________
softmax_26 (Softmax)         (None, 2)                 0         
Total params: 14,338
Trainable params: 14,338
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
32/32 [==============================] - 1s 17ms/step - loss: 0.3667 - auc: 0.9259 - val_loss: 0.3093 - val_auc: 0.9437
Epoch 2/30
32/32 [==============================] - 0s 5ms/step - loss: 0.2816 - auc: 0.9531 - val_loss: 0.3214 - va

Wyniki nie sa lepsze niz gdy korzystalismy z XGboosta, spróbujmy poradzic cos na nasze niezbalansowanie w zbiorze.

In [59]:
# skorzystajmy z dostepnej metody z dedykowanej biblioteki
smote = SMOTE(sampling_strategy = 'minority',random_state=1)
x_sm, y_sm = smote.fit_resample(X,Y)


# podzielmy jeszcze raz zbior na zbior treningowy i testowy
X_train1, X_test1, Y_train1,Y_test1 = train_test_split(x_sm,y_sm, test_size = 0.2)

In [60]:
#jak pamietamy nasz data set był niezbalansowany sprawdzimy czy oversampling poprawił nam udziały klasy.
Y_train1.value_counts()

1    1120
2    1091
Name: quality_category, dtype: int64

In [61]:
# Przetworzmy dane do formatu wejsciowego do modelu

X_train_array_d = X_train1.values
X_test_array_d = X_test1.values


# transformujemy Y_test_duplicated i Y_test_duplicated

Y_train_d = np.array(Y_train1)
Y_test_d = np.array(Y_test1)

Y_test_d = OneHotEncoder().fit_transform(Y_test_d.reshape(-1,1)).toarray()
# zmienmy jeszcze format
Y_test_d = Y_test_d.astype(np.float32)

# transformujemy y_train
Y_train_d = OneHotEncoder().fit_transform(Y_train_d.reshape(-1,1)).toarray()
# zmienmy jeszcze format
Y_train_d = Y_train_d.astype(np.float32)

In [70]:
#sprawdzmy jak z przerobionym zestawem danych dziala nasza siec neuronowa
model_nn3 = model_nn(X_train_array_d, Y_train_d,[128],0.1,2,30)
Y_pred_nn3 = model_nn3.predict(X_test_array_d).argmax(axis = 1)

# szybka ewaluacja
eval_res = model_nn3.evaluate(X_test_array_d, Y_test_d)
eval_names = model_nn3.metrics_names
print(f"{eval_names[0]}: {eval_res[0]}")
print(f"{eval_names[1]}: {eval_res[1]}")

#obejrzyjmy jeszcze macierz pomyłek 
matrix_nn3 = confusion_matrix(Y_test_d.argmax(axis = 1), Y_pred_nn3)
raport_nn3 = classification_report(Y_test_d.argmax(axis = 1), Y_pred_nn3)
print(raport_nn3)

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ukryta_0 (Dense)             (None, 128)               1536      
_________________________________________________________________
dropout_76 (Dropout)         (None, 128)               0         
_________________________________________________________________
WYJSCIE (Dense)              (None, 2)                 258       
_________________________________________________________________
softmax_45 (Softmax)         (None, 2)                 0         
Total params: 1,794
Trainable params: 1,794
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
56/56 [==============================] - 2s 12ms/step - loss: 1.1632 - auc: 0.6060 - val_loss: 0.6232 - val_auc: 0.7271
Epoch 2/30
56/56 [==============================] - 0s 3ms/step - loss: 0.6491 - auc: 0.7209 - val_loss: 0.6454 - val_

Nasz model znacznie poprawil rezultaty dla drugiej klasy, ale przez to spadla dokladnosc dla klasy, ktora pierwotnie byla mniej liczna. 

In [116]:
#sprobujmy jeszcze inne konfiguracje
model_nn4 = model_nn(X_train_array_d, Y_train_d,[256],0.0,2,30)
Y_pred_nn4 = model_nn4.predict(X_test_array_d).argmax(axis = 1)

# szybka ewaluacja
eval_res = model_nn4.evaluate(X_test_array_d, Y_test_d)
eval_names = model_nn4.metrics_names
print(f"{eval_names[0]}: {eval_res[0]}")
print(f"{eval_names[1]}: {eval_res[1]}")

#obejrzyjmy jeszcze macierz pomyłek 
matrix_nn4 = confusion_matrix(Y_test_d.argmax(axis = 1), Y_pred_nn4)
raport_nn4 = classification_report(Y_test_d.argmax(axis = 1), Y_pred_nn4)
print(raport_nn4)

Model: "sequential_89"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ukryta_0 (Dense)             (None, 256)               3072      
_________________________________________________________________
dropout_132 (Dropout)        (None, 256)               0         
_________________________________________________________________
WYJSCIE (Dense)              (None, 2)                 514       
_________________________________________________________________
softmax_89 (Softmax)         (None, 2)                 0         
Total params: 3,586
Trainable params: 3,586
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
56/56 [==============================] - 1s 9ms/step - loss: 1.5363 - auc: 0.6183 - val_loss: 0.5947 - val_auc: 0.7553
Epoch 2/30
56/56 [==============================] - 0s 3ms/step - loss: 0.5828 - auc: 0.7665 - val_loss: 0.6899 - val_a

Nasz najlepszy model uzyskał całkiem dobry wynik dla obu klas, zapiszmy jego wynik.

In [120]:
model_nn4.save("modele/model_nn4.h5")